Questo programma calcola la likelihood poissoniana associata al numero di ammassi di galassia ad un certo redshift e con una certa massa.

Importo le funzioni utili allo svolgimento del lavoro:

In [ ]:
import numpy as np
#import pylab as plt
import pyccl as ccl
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13
from astropy.cosmology import FlatLambdaCDM as FLCDM
%matplotlib inline

Definisco una funzione che mi permette di inserire solo i label al centro del bin, nei plot:

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = 0.025
    plt.xticks(np.arange(min(bins)+bin_w, max(bins)+bin_w, 2*bin_w), bins, **kwargs)

Leggo i dati dalla tabella cat.txt (formata da due colonne), e li assegno a due array, M e z:

In [ ]:
GCdata = np.loadtxt('cat.txt', usecols=(0,1))
#print(GCdata.shape)

M = GCdata[:,0]
z = GCdata[:,1]

Definisco la cosmologia di riferimento per il calcolo del valore teorico del numero di ammassi; la cosmologia è quella di Planck 13, uguale a quella dei dati osservativi. Definisco il binning in redshift e massa

In [ ]:
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96)
cs = ccl.halos.MassFuncPress74(cosmo)

z_bin = np.arange(0.05, 2.15, 0.1)
#print(z_bin)
M_bin = np.arange(14.025, 15.525, 0.05)
#print(M_bin)

Doppio ciclo for per calcolare la likelihhod per ogni redshift. Il plot finale mostra la log likelihood al variare del redshift. E' possibile abilitare la visualizzazione dei plot intermedi per ogni redshift, dove viene mostrato, a z fissato, l'istogramma del numero di conteggi, con sovraplottato il valore teorico del numero di ammassi, per ogni bin di massa.

In [ ]:
bins = np.round(np.arange(14,15.6,0.05),3)

i = 0
k = 0
log_l = []
for red in z_bin:
    M_z = M[(z>=red-0.05) & (z<red+0.05)]
    plt.figure()
    n_obs, bins_hist, patches = plt.hist(M_z, bins, edgecolor = 'r', facecolor = 'red', alpha = 0.2, label = 'obs')
    
    N = []
    log_p = []
    j = 0
    for mass in M_bin:
        ind_exp = np.arange(mass-0.025, mass+0.025,0.005)
        masses = 10**ind_exp
        nm = cs.get_mass_function(cosmo, masses, 1/(1+red))
        v_min = Planck13.comoving_volume(red-0.05).value 
        v_max = Planck13.comoving_volume(red+0.05).value 
        v_shell = (v_max-v_min)*0.36
        #print(v_max, v_min, v_shell)
        Nm = nm*v_shell
        k = np.trapz(Nm, x = ind_exp)
        N.append(k)
        log_p.append(n_obs[j]*np.log(k)-k)
        j += 1
    log_l.append(np.sum(log_p))  
    #bins_labels(bins, fontsize = 10)
    plt.scatter(M_bin, N, marker = '.', label='theory')
    plt.xlabel('log(M)')
    plt.ylabel('Number counts')
    plt.title('%.2f<z<%.2f'%(red-0.05,red+0.05))
    plt.legend()
    #plt.savefig('mass_distr_%d.png' %i, bbox_inches = 'tight')
    plt.close()
    i += 1
#print(log_l)
plt.scatter(z_bin, log_l)
plt.xlabel('z')
plt.ylabel('Log(L)')